In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander
using DiffResults
using QuadGK

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


### Basis

In [4]:
Nx = 6
B = CstProHermite(7)
MB = MultiBasis(B, Nx)

MultiBasis{9,6}(Basis of 9 functions:
ProPolyHermite{0}(ImmutablePolynomial(1.0), false)
ProHermite{0}(ProPolyHermite{0}(ImmutablePolynomial(1.0), false), false)
ProHermite{1}(ProPolyHermite{1}(ImmutablePolynomial(1.0*x), false), false)
ProHermite{2}(ProPolyHermite{2}(ImmutablePolynomial(-1.0 + 1.0*x^2), false), false)
ProHermite{3}(ProPolyHermite{3}(ImmutablePolynomial(-3.0*x + 1.0*x^3), false), false)
ProHermite{4}(ProPolyHermite{4}(ImmutablePolynomial(3.0 - 6.0*x^2 + 1.0*x^4), false), false)
ProHermite{5}(ProPolyHermite{5}(ImmutablePolynomial(15.0*x - 10.0*x^3 + 1.0*x^5), false), false)
ProHermite{6}(ProPolyHermite{6}(ImmutablePolynomial(-15.0 + 45.0*x^2 - 15.0*x^4 + 1.0*x^6), false), false)
ProHermite{7}(ProPolyHermite{7}(ImmutablePolynomial(-105.0*x + 105.0*x^3 - 21.0*x^5 + 1.0*x^7), false), false)
)

f (generic function with 1 method)

In [15]:
@btime ForwardDiff.gradient(x->B[5](x[1]),[5.0])

  1.876 μs (9 allocations: 480 bytes)


1-element Array{Float64,1}:
 -0.39188218965422494

In [12]:
@btime ForwardDiff.derivative(x->B[5](x),5.0)

  115.295 ns (6 allocations: 176 bytes)


-0.39188218965422494

In [16]:
function timing()
    f(x) = (x^5-4*x^3+1)*exp(-x^2/2)  
    @btime ForwardDiff.gradient(x->f(x[1]),[5.0])
    @btime ForwardDiff.derivative(x->f(x),5.0)
end

timing (generic function with 1 method)

In [17]:
timing()

  373.897 ns (4 allocations: 320 bytes)
  90.438 ns (0 allocations: 0 bytes)


-0.0384031609382707

In [3]:
@btime quadgk(x->FamilyPhyHermite[5](x), 0, 1.0)

LoadError: UndefVarError: @btime not defined

In [7]:
Nx = 10
Nψ = 8
B = CstProHermite(4)
MB = MultiBasis(B, Nx)
f = ExpandedFunction([MultiFunction(MB, rand(1:6,10)) for i=1:5], randn(5));

In [15]:
result = DiffResults.HessianResult(randn(10));
@time result = ForwardDiff.gradient!(result, f, randn(10));

  0.000043 seconds (240 allocations: 23.766 KiB)


In [ ]:
first()

In [3]:
function timing2()
    @btime x = randn(100,20)
    @btime begin 
        x = randn(20,200)
        for i=1:200
        x[10,i]
        end
    end
end

timing2 (generic function with 1 method)

In [4]:
timing2()

  8.618 μs (1 allocation: 15.75 KiB)
  17.632 μs (2 allocations: 31.33 KiB)


In [7]:
function timing()
    Nx = 10
    Nψ = 8
    B = CstProHermite(4)
    MB = MultiBasis(B, Nx);
    f = ExpandedFunction([MultiFunction(MB, rand(1:6,10)) for i=1:5], randn(5));
    [f(randn(10)) for i=1:200]
end

timing (generic function with 1 method)

In [8]:
@btime timing()

  768.506 μs (41681 allocations: 684.45 KiB)


200-element Array{Float64,1}:
 -8.590225160417804
 -0.5239976826059032
  6.66825240739424
  0.5126861115528971
  1.1890170956366577
  1.9099015433839461
 -2.6208237682305615
 -0.2750170730292265
  1.9334084408087548
  3.109658768387867
  2.53494608597899
 -3.546217671466872
 -1.268588739020128
  ⋮
  0.16818086051151243
 -0.5800634012353066
  3.1511064017310857
 -0.005207920621169093
  3.0224252468487416
 -1.2766993119159948
 -2.455229445475138
  2.3197792742642527
  0.25028959373953996
 -2.4814389112726265
  0.5588625689546698
 -0.6431622213693868

In [23]:
@time [f(randn(10)) for i=1:200]

  0.059839 seconds (132.30 k allocations: 5.013 MiB)


200-element Array{Float64,1}:
  0.3565862149179235
 -0.5592502129695079
  3.4809269148513504
  1.804493123230571
 -1.1765146752143598
 -0.8935079670961806
  2.5101693581610216
  0.6623619999574702
  0.052252195957035855
 -0.4985407749185703
  2.4338245167454438
 -3.7831461327458253
 -0.055680428933647226
  ⋮
 -3.504190540995787
  2.6387529842618522
 -0.049235990854431796
 -0.015306426123755129
  0.2696121853577881
  0.9389268368122676
 -0.4961688355677305
  2.581397667362235
  2.7790344221281584
  0.3852747164672356
 -3.5747482662361025
 -3.4335180911532364

In [28]:
@btime ForwardDiff.gradient(f, randn(10))

  3.055 μs (50 allocations: 6.41 KiB)


10-element Array{Float64,1}:
  0.0
 -0.41481567821516313
  0.0
 -0.2054768734405355
 -0.1616353245698944
  0.06113183193954419
  0.06236459956882944
  0.02235240578579206
  0.16626206776483202
  0.3377408008797908

In [38]:
function timing() 
    f = MultiFunction(MB, rand(1:6,10));
    result = DiffResults.HessianResult(randn(10));
    result = ForwardDiff.hessian!(result, f, randn(10));
end

timing (generic function with 1 method)

In [42]:
function timing2() 
    f = MultiFunction(MB, rand(1:6,10));
    x = randn(10)
    f(x)
    ForwardDiff.gradient(f, x);
    ForwardDiff.hessian(f, x);
end

timing2 (generic function with 1 method)

In [43]:
@btime timing()

  6.722 μs (47 allocations: 51.77 KiB)


MutableDiffResult(-0.2044578068550304, ([0.39829120904469095, -0.19262095275384977, -0.06467058375300636, -0.16597678410144642, 0.09076585820500507, -0.4505810392251209, -0.10561444535867072, 0.12783878526911294, -0.11042588440504841, 0.10070902078475591], [0.627576166543466 0.3752325887661919 0.12598063820020433 0.32332878372302787 -0.17681517747122896 0.8777481752645109 0.20574076275950454 -0.24903458141728996 0.21511362019402344 -0.1961848181151047; 0.3752325887661919 0.0 -0.060926553254502115 -0.15636774540631362 0.08551107122735313 -0.4244951582108158 -0.09950001617683905 0.12043770299703523 -0.1040329023868896 0.09487858562535327; 0.12598063820020433 -0.060926553254502115 0.08177341450641945 -0.052498927198695755 0.02870949818571752 -0.14252005967855214 -0.0334061483841412 0.04043577008282555 -0.03492802019039548 0.03185454868916674; 0.32332878372302787 -0.15636774540631362 -0.052498927198695755 0.5065202209408731 0.07368280763060513 -0.36577714012505186 -0.08573674082163721 0.10

In [44]:
@btime timing2()

  12.227 μs (74 allocations: 46.17 KiB)


10×10 Array{Float64,2}:
  0.0        0.121232    0.0   0.656108   …  0.0  -0.856951   0.0   1.31124
  0.121232  -0.505286    0.0   0.0285255     0.0  -0.0372576  0.0   0.0570086
  0.0        0.0         0.0   0.0           0.0   0.0        0.0   0.0
  0.656108   0.0285255   0.0  -0.487653      0.0  -0.201639   0.0   0.308532
  0.21697    0.00943319  0.0   0.0510526     0.0  -0.0666804  0.0   0.102029
 -0.12933   -0.00562285  0.0  -0.0304309  …  0.0   0.0397462  0.0  -0.0608166
  0.0        0.0         0.0   0.0           0.0   0.0        0.0   0.0
 -0.856951  -0.0372576   0.0  -0.201639      0.0  -0.376055   0.0  -0.402977
  0.0        0.0         0.0   0.0           0.0   0.0        0.0   0.0
  1.31124    0.0570086   0.0   0.308532      0.0  -0.402977   0.0   0.0

### Hermite functions

In [27]:
@time integral, err = quadgk(x -> FamilyPhyHermite[5](x), 0.0, 1.0, rtol=1e-8)

  0.069184 seconds (167.45 k allocations: 8.917 MiB)


(0.5630021473036511, 1.4497070210950369e-11)

In [29]:
derivative(FamilyPhyHermite[5],-1, [1.0])

1-element Array{Float64,1}:
 -18.84597896350062

In [88]:
@testset "Verify second derivative of Physicist hermite functions" begin
    # Unscaled
    x = randn(100)
    m = 5
    dV = vander(PhyHermite(m; scaled = false), 2, x)

    for i=0:5
        Hn = FamilyPhyPolyHermite[i+1].(x)
        Hnp = derivative(FamilyPhyPolyHermite[i+1],1).(x)
        Hnpp = derivative(FamilyPhyPolyHermite[i+1],2).(x)
        E = exp.(-x.^2/2)

        @test norm(dV[:,i+1] - (Hnpp .*E -2 .*x .* Hnp.*E + Hn .* (x.^2 .- 1.0) .* E))<1e-8
    end

    # Scaled

    x = randn(100)
    m = 5
    dV = vander(PhyHermite(m; scaled = true), 2, x)

    for i=0:5
        Hn = FamilyPhyPolyHermite[i+1].(x)
        Hnp = derivative(FamilyPhyPolyHermite[i+1],1).(x)
        Hnpp = derivative(FamilyPhyPolyHermite[i+1],2).(x)
        E = exp.(-x.^2/2)

        @test norm(dV[:,i+1] - (1/Cphy(i))*(Hnpp .*E -2 .*x .* Hnp.*E + Hn .* (x.^2 .- 1.0) .* E))<1e-8
    end
end

Test Summary:                                           | Pass  Total
Verify second derivative of Physicist hermite functions |   12     12


Test.DefaultTestSet("Verify second derivative of Physicist hermite functions", Any[], 12, false)